In [1]:
import os
from openai import OpenAI
import tomllib  # estándar en Python 3.11+
from pathlib import Path
from datetime import date

#  Credenciales OpenRouter.ai (OpenAI compatible)
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
    default_headers={               # opcional, recomendado
        "HTTP-Referer": "http://localhost",
        "X-Title": "IA_generativa Notebooks",
    },
)

# listar algunos modelos disponibles
models = client.models.list()
print("Modelos disponibles:", len(models.data))
for m in models.data[:10]:
    print("-", m.id)

Modelos disponibles: 326
- thedrummer/cydonia-24b-v4.1
- relace/relace-apply-3
- google/gemini-2.5-flash-preview-09-2025
- google/gemini-2.5-flash-lite-preview-09-2025
- qwen/qwen3-vl-235b-a22b-thinking
- qwen/qwen3-vl-235b-a22b-instruct
- qwen/qwen3-max
- qwen/qwen3-coder-plus
- openai/gpt-5-codex
- deepseek/deepseek-v3.1-terminus


## Carga y procesamiento de la base de datos

In [2]:
project_root = Path.cwd()
data = project_root / "ecomarket_base_datos.toml"

data_path = Path(data)
with data_path.open("rb") as data_file:
    documento = tomllib.load(data_file)
    

def _to_date_str(v):
  
    if hasattr(v, "isoformat"):
        return v.isoformat()
    return str(v)

def build_orders_document(orders_list):
    """
    Convierte la lista de pedidos en el bloque de texto esperado por tu prompt.
    """
    lines = [
        "# DOCUMENTO_DE_PEDIDOS — EcoMarket",
        "# - <tracking>: status='<status>', eta='<YYYY-MM-DD>', carrier='<carrier>', url='<url>',",
        "#   delayed=<true|false>, order_id='<id>', cliente='<nombre>', ciudad='<ciudad>', producto='<producto>'",
        "",
    ]
    for o in orders_list:
        lines.append(
            f"- {o['tracking_number']}: status='{o['status']}', eta='{_to_date_str(o['eta'])}', "
            f"carrier='{o['carrier']}', url='{o['track_url']}', delayed={str(o['delayed']).lower()}, "
            f"order_id='{o['order_id']}', cliente='{o['customer_name']}', ciudad='{o['city']}', "
            f"producto='{o['product']}'"
        )
    return "\n".join(lines)

orders = documento["orders"]               
documento_txt = build_orders_document(orders)  
print(documento_txt.splitlines()[:6])


['# DOCUMENTO_DE_PEDIDOS — EcoMarket', "# - <tracking>: status='<status>', eta='<YYYY-MM-DD>', carrier='<carrier>', url='<url>',", "#   delayed=<true|false>, order_id='<id>', cliente='<nombre>', ciudad='<ciudad>', producto='<producto>'", '', "- 20001: status='Procesando', eta='2025-10-02', carrier='DHL', url='https://tracking.ecomarket.example/dhl/20001', delayed=false, order_id='ECO-2509-001', cliente='Ana Pérez', ciudad='Bogotá', producto='Botella reutilizable de acero'", "- 20002: status='En preparación', eta='2025-10-03', carrier='UPS', url='https://tracking.ecomarket.example/ups/20002', delayed=false, order_id='ECO-2509-002', cliente='Luis Gómez', ciudad='Medellín', producto='Cepillo dental de bambú'"]


## Cargar Settings

In [3]:
settings_path = Path(project_root / "ecomarket_settings_final.toml")
with settings_path.open("rb") as settings_path:
    settings = tomllib.load(settings_path)
print(settings)

{'general': {'model': 'meta-llama/llama-3.3-70b-instruct:free', 'temperature': 0.2}, 'prompts': {'role_prompt': 'Eres un agente de servicio al cliente de EcoMarket: amable, empático y conciso.\nReglas generales:\n- Detecta la intención del usuario: {seguimiento_de_pedido | devolucion_de_producto}.\n- Usa EXCLUSIVAMENTE las fuentes provistas:\n  • DOCUMENTO_DE_PEDIDOS para estados/ETAs/links/carrier.\n  • POLITICA_DEVOLUCIONES para elegibilidad y proceso de devoluciones.\n- Si falta información (número de seguimiento/pedido, fecha de entrega, motivo), pídela amablemente.\n- No inventes datos; responde en español claro, 3–6 líneas.\n', 'instruction_prompt': 'DOCUMENTO_DE_PEDIDOS:\n>>>>>DOC_PEDIDOS<<<<<\n\nPOLITICA_DEVOLUCIONES:\n>>>>>DOC_POLITICA<<<<<\n\nMENSAJE_DEL_USUARIO:\n>>> {{user_utterance}} <<<\n\nTarea:\n1) Identifica intención:\n   - Seguimiento: localiza el tracking en el mensaje; si existe en el documento, devuelve estado, ETA, link y carrier. Si delayed=true, disculpa breve 

In [4]:
devolucion_path = Path(project_root / "politicas_devoluciones.toml")
with devolucion_path.open("rb") as devolucion_file:
    devolucion = tomllib.load(devolucion_file)
print(devolucion)

{'policy': {'company': 'EcoMarket', 'window_days': 30, 'doa_days': 7, 'refund_days': '5–10', 'requirements': ['Empaque original', 'Producto sin uso', 'Comprobante de compra'], 'allowed_categories': ['Hogar', 'Cocina', 'Energía', 'Limpieza'], 'non_returnable': ['Perecederos', 'Higiene/cosmética abiertos', 'Ropa interior usada', 'Productos personalizados'], 'steps': ['Solicitar número de seguimiento/pedido, fecha de entrega y motivo', 'Validar elegibilidad según la política', 'Generar etiqueta, empacar y entregar en el punto indicado', 'Procesar reembolso en 5–10 días hábiles tras recepción'], 'alternatives': ['Soporte de uso', 'Cambio por artículo elegible', 'Guía de reciclaje responsable']}}


In [5]:
def _as_list(x):
    if x is None:
        return []
    return x if isinstance(x, list) else [x]

def build_policy_text_from_toml(devolucion: dict) -> str:
    """
    Convierte el dict cargado de politicas_devoluciones.toml en el bloque POLITICA_DEVOLUCIONES
    que tu prompt espera (texto conciso y estructurado).
    """
    pol = devolucion.get("policy", devolucion)  # por si el archivo no tiene sección [policy]
    company       = pol.get("company", "EcoMarket")
    window_days   = pol.get("window_days", 30)
    doa_days      = pol.get("doa_days", 7)
    refund_days   = pol.get("refund_days", "5–10")

    requirements       = _as_list(pol.get("requirements"))
    allowed_categories = _as_list(pol.get("allowed_categories"))
    non_returnable     = _as_list(pol.get("non_returnable"))
    steps              = _as_list(pol.get("steps"))
    alternatives       = _as_list(pol.get("alternatives"))

    lines = [
        "# POLITICA_DEVOLUCIONES — " + company,
        f"VENTANA_STANDARD: {window_days} días desde la entrega. DOA: reportado dentro de {doa_days} días.",
        f"Reembolso: {refund_days} días hábiles tras recepción.",
        "REQUISITOS: " + (", ".join(requirements) if requirements else "–"),
        "ELEGIBLES (ejemplos): " + (", ".join(allowed_categories) if allowed_categories else "–"),
        "NO ELEGIBLES: " + (", ".join(non_returnable) if non_returnable else "–"),
        "PROCESO:",
    ]
    # pasos
    for i, s in enumerate(steps, 1):
        lines.append(f"{i}) {s}")
    if alternatives:
        lines.append("ALTERNATIVAS: " + ", ".join(alternatives))

    return "\n".join(lines)

In [6]:
POLITICA_DEVOLUCIONES_TXT = build_policy_text_from_toml(devolucion)
print(POLITICA_DEVOLUCIONES_TXT.splitlines()[:8])  # vista rápida

['# POLITICA_DEVOLUCIONES — EcoMarket', 'VENTANA_STANDARD: 30 días desde la entrega. DOA: reportado dentro de 7 días.', 'Reembolso: 5–10 días hábiles tras recepción.', 'REQUISITOS: Empaque original, Producto sin uso, Comprobante de compra', 'ELEGIBLES (ejemplos): Hogar, Cocina, Energía, Limpieza', 'NO ELEGIBLES: Perecederos, Higiene/cosmética abiertos, Ropa interior usada, Productos personalizados', 'PROCESO:', '1) Solicitar número de seguimiento/pedido, fecha de entrega y motivo']


In [7]:
def assemble_messages_unified(prompts: dict, doc_pedidos: str, doc_politica: str, user_utterance: str):
    P = prompts
    instr = (
        P["instruction_prompt"]
        .replace(">>>>>DOC_PEDIDOS<<<<<", doc_pedidos)
        .replace(">>>>>DOC_POLITICA<<<<<", doc_politica)
        .replace("{{user_utterance}}", user_utterance)
    )

    # Heurística simple para elegir qué few-shots añadir primero (opcional)
    u = user_utterance.lower()
    is_returns = any(k in u for k in [
        "devoluci", "devolver", "cambio", "reembolso", "refund", "de vuelta"
    ])

    msgs = [{"role": "system", "content": P["role_prompt"]}]

    if is_returns:
        # Prioriza ejemplos de devoluciones + uno de seguimiento por si el usuario mezcla
        msgs += [
            {"role": "user", "content": P["return_positive_example"]},
            {"role": "system", "content": P["return_positive_reasoning"]},
            {"role": "assistant", "content": P["return_positive_output"]},

            {"role": "user", "content": P["return_negative_example"]},
            {"role": "system", "content": P["return_negative_reasoning"]},
            {"role": "assistant", "content": P["return_negative_output"]},

            {"role": "user", "content": P["track_negative_example"]},
            {"role": "system", "content": P["track_negative_reasoning"]},
            {"role": "assistant", "content": P["track_negative_output"]},
        ]
    else:
        # Prioriza ejemplos de seguimiento + uno de devoluciones por si la intención cambia
        msgs += [
            {"role": "user", "content": P["track_negative_example"]},
            {"role": "system", "content": P["track_negative_reasoning"]},
            {"role": "assistant", "content": P["track_negative_output"]},

            {"role": "user", "content": P["track_positive_example"]},
            {"role": "system", "content": P["track_positive_reasoning"]},
            {"role": "assistant", "content": P["track_positive_output"]},

            {"role": "user", "content": P["return_positive_example"]},
            {"role": "system", "content": P["return_positive_reasoning"]},
            {"role": "assistant", "content": P["return_positive_output"]},
        ]

    msgs.append({"role": "user", "content": instr})
    return msgs

In [8]:
def ask_support_unified(doc_pedidos_txt: str, politica_txt: str, user_utterance: str, client, SETTINGS):
    msgs = assemble_messages_unified(SETTINGS["prompts"], doc_pedidos_txt, politica_txt, user_utterance)
    resp = client.chat.completions.create(
        model=SETTINGS["general"]["model"],
        messages=msgs,
        temperature=SETTINGS["general"]["temperature"],
        max_tokens=400,
    )
    return resp.choices[0].message.content

In [9]:
# 1) Solo saludo → debe saludar y pedir el número de seguimiento.
print(ask_support_unified(
    documento_txt, POLITICA_DEVOLUCIONES_TXT,
    "Hola 👋 ¿me ayudas con el estado de mi pedido?",
    client, settings
))
print("-----")

# 2) Saludo sin contexto (sin tracking) → debe pedir el número amablemente.
print(ask_support_unified(
    documento_txt, POLITICA_DEVOLUCIONES_TXT,
    "Hola 👋 jairo",
    client, settings
))
print("-----")

# 3) Tracking explícito válido (ej. 20004) → debe devolver estado, ETA, link y carrier.
print(ask_support_unified(
    documento_txt, POLITICA_DEVOLUCIONES_TXT,
    "Buenas, el número es 20004. ¿Cómo va?",
    client, settings
))
print("-----")

# 4) Varios números; el modelo debe elegir el que exista en el documento (ej. 20003).
print(ask_support_unified(
    documento_txt, POLITICA_DEVOLUCIONES_TXT,
    "Tengo 12345 y 20003, creo que es el segundo. ¿Me confirmas el estado?",
    client, settings
))
print("-----")

# 5) Número inexistente (ej. 99999) → debe responder con empatía y pedir confirmación/corrección.
print(ask_support_unified(
    documento_txt, POLITICA_DEVOLUCIONES_TXT,
    "Estado del pedido 99999, por favor.",
    client, settings
))
print("-----")

¡Hola! Para ayudarte con el estado de tu pedido, necesito el número de seguimiento. ¿Podrías proporcionarlo, por favor? Estoy aquí para ayudarte.
-----
¡Hola Jairo! ¿En qué puedo ayudarte hoy? ¿Tienes algún número de seguimiento o pedido que quieras consultar?
-----
El pedido 20004 está "Retrasado". La fecha estimada de entrega es 2025-10-03, y el carrier es Servientrega. Puedes rastrearlo en https://tracking.ecomarket.example/servientrega/20004. Lamentamos la demora; el transportista reporta alta demanda. Te ofrecemos un 10% de descuento en tu próxima compra. ¿Necesitas algo más?
-----
El pedido 20003 está "En tránsito". Fecha estimada de entrega: 2025-09-28, transportadora=FedEx. Puedes rastrearlo en https://tracking.ecomarket.example/fedex/20003. ¿Necesitas ayuda con algo más?
-----
Lo siento, pero no tengo información sobre el pedido 99999. ¿Podrías proporcionarme el número de seguimiento correcto para ayudarte con el estado de tu pedido?
-----


In [10]:
# 1) Elegible (sin abrir, dentro de 30 días) → debe pedir nº de pedido/seguimiento y dar pasos
print(ask_support_unified(documento_txt, POLITICA_DEVOLUCIONES_TXT,
      "Hola, quiero devolver una bolsa compostable sin abrir; llegó hace 10 días. ¿Cómo hago?",
      client, settings))
print("-----")

# 2) No elegible (higiene/cosmética abierto) → explicar con empatía y ofrecer alternativa
print(ask_support_unified(documento_txt, POLITICA_DEVOLUCIONES_TXT,
      "Quiero devolver un perfume que abrí y no me gustó.",
      client, settings))
print("-----")

# 3) DOA / Daño en transporte (reportado dentro de 7 días) → solicitar evidencias y dar pasos de reemplazo/reembolso
print(ask_support_unified(documento_txt, POLITICA_DEVOLUCIONES_TXT,
      "El panel solar portátil llegó golpeado hoy. ¿Puedo devolverlo o cambiarlo?",
      client, settings))
print("-----")

# 4) No elegible (perecedero) → rechazar con empatía y alternativa
print(ask_support_unified(documento_txt, POLITICA_DEVOLUCIONES_TXT,
      "Compré café en grano (perecedero) y lo abrí; quiero devolverlo.",
      client, settings))
print("-----")

# 5) Fuera de ventana (pasados 30 días, sin usar) → explicar límite y alternativas
print(ask_support_unified(documento_txt, POLITICA_DEVOLUCIONES_TXT,
      "Necesito devolver un detergente ecológico sin usar, pero ya pasaron 45 días desde la entrega.",
      client, settings))
print("-----")

La bolsa compostable es elegible para devolución. ¿Podrías proporcionarme el número de pedido para generar la etiqueta de devolución? Recuerda que debe estar en su empaque original y sin abrir. Te guiaré con los pasos siguientes.
-----
Lamento que el perfume no te haya gustado. Desafortunadamente, los productos de higiene/cosmética abiertos no son elegibles para devolución. Te ofrezco soporte de uso o una guía de reciclaje responsable como alternativa. ¿Te gustaría explorar alguna de estas opciones?
-----
Lamento que el panel solar portátil haya llegado dañado. Para proceder, necesito el número de seguimiento o pedido. ¿Podrías proporcionarlo? También necesitaré fotos del daño para validar el incidente y proceder con la devolución o cambio.
-----
Lamento que no te guste el café en grano. Desafortunadamente, los productos perecederos no son elegibles para devolución. Te ofrezco soporte de uso o una guía de reciclaje responsable para ayudarte a disfrutar de tu compra de manera sostenible

In [ ]:
def assemble_messages_unified(prompts: dict, doc_pedidos: str, doc_politica: str, user_utterance: str):
    P = prompts
    instr = (
        P["instruction_prompt"]
        .replace(">>>>>DOC_PEDIDOS<<<<<", doc_pedidos)
        .replace(">>>>>DOC_POLITICA<<<<<", doc_politica)
        .replace("{{user_utterance}}", user_utterance)
    )

    # Heurística simple para elegir qué few-shots añadir primero (opcional)
    u = user_utterance.lower()
    is_returns = any(k in u for k in [
        "devoluci", "devolver", "cambio", "reembolso", "refund", "de vuelta"
    ])

    msgs = [{"role": "system", "content": P["role_prompt"]}]

    if is_returns:
        # Prioriza ejemplos de devoluciones + uno de seguimiento por si el usuario mezcla
        msgs += [
            {"role": "user", "content": P["return_positive_example"]},
            {"role": "system", "content": P["return_positive_reasoning"]},
            {"role": "assistant", "content": P["return_positive_output"]},

            {"role": "user", "content": P["return_negative_example"]},
            {"role": "system", "content": P["return_negative_reasoning"]},
            {"role": "assistant", "content": P["return_negative_output"]},

            {"role": "user", "content": P["track_negative_example"]},
            {"role": "system", "content": P["track_negative_reasoning"]},
            {"role": "assistant", "content": P["track_negative_output"]},
        ]
    else:
        # Prioriza ejemplos de seguimiento + uno de devoluciones por si la intención cambia
        msgs += [
            {"role": "user", "content": P["track_negative_example"]},
            {"role": "system", "content": P["track_negative_reasoning"]},
            {"role": "assistant", "content": P["track_negative_output"]},

            {"role": "user", "content": P["track_positive_example"]},
            {"role": "system", "content": P["track_positive_reasoning"]},
            {"role": "assistant", "content": P["track_positive_output"]},

            {"role": "user", "content": P["return_positive_example"]},
            {"role": "system", "content": P["return_positive_reasoning"]},
            {"role": "assistant", "content": P["return_positive_output"]},
        ]

    msgs.append({"role": "user", "content": instr})
    return msgs


In [4]:
def assemble_messages(prompts: dict, documento: str, user_utterance: str):
    P = prompts
    instr = (
        P["instruction_prompt"]
        .replace(">>>>>DOCUMENTO<<<<<", documento)
        .replace("{{user_utterance}}", user_utterance)
    )
    return [
        {"role": "system", "content": P["role_prompt"]},
        {"role": "user", "content": P["negative_example"]},
        {"role": "system", "content": P["negative_reasoning"]},
        {"role": "assistant", "content": P["negative_output"]},
        {"role": "user", "content": P["positive_example"]},
        {"role": "system", "content": P["positive_reasoning"]},
        {"role": "assistant", "content": P["positive_output"]},
        {"role": "user", "content": instr},
    ]
    
def ask_status_infer(documento_txt: str, user_utterance: str, client, SETTINGS):
    msgs = assemble_messages(SETTINGS["prompts"], documento_txt, user_utterance)
    resp = client.chat.completions.create(
        model=SETTINGS["general"]["model"],
        messages=msgs,
        temperature=SETTINGS["general"]["temperature"],
    )
    return resp.choices[0].message.content


# 1) Solo saludo (debe pedir el tracking)
print(ask_status_infer(documento_txt, "Hola 👋 ¿me ayudas con el estado de mi pedido?", client, settings))
print("-----")

print(ask_status_infer(documento_txt, "Hola 👋 jairo", client, settings))
print("-----")

# 2) Mensaje con un número válido existente en tu documento (p. ej., 20004)
print(ask_status_infer(documento_txt, "Buenas, el número es 20004. ¿Cómo va?", client, settings))
print("-----")

# 3) Mensaje con varios números (uno válido en doc, otro no)
print(ask_status_infer(documento_txt, "Tengo 12345 y 20003, creo que es el segundo. ¿Me confirmas el estado?", client, settings))
print("-----")

# 4) Número que no existe en el documento
print(ask_status_infer(documento_txt, "Estado del pedido 99999, por favor.", client, settings))

¡Hola! Gracias por escribirnos. Para ayudarte con el estado de tu pedido, ¿me compartes el número de seguimiento? Si no lo tienes a mano, puedes encontrarlo en el correo de confirmación.
-----
¡Hola Jairo! 👋 Gracias por escribirnos. Para ayudarte con el estado de tu pedido, ¿me compartes el número de seguimiento? Si no lo tienes a mano, puedes encontrarlo en el correo de confirmación.
-----
El pedido 20004 está "Retrasado" con fecha estimada de entrega 2025-10-03. Puedes rastrearlo en https://tracking.ecomarket.example/servientrega/20004 con Servientrega. Lamentamos la demora. Te ofrecemos un 10% de descuento en tu próxima compra.
-----
El pedido 20003 está "En tránsito" con fecha estimada de entrega 2025-09-28. Puedes rastrearlo en https://tracking.ecomarket.example/fedex/20003 con FedEx. ¿Necesitas más ayuda?
-----
Lo siento, el número de seguimiento 99999 no se encuentra en nuestro registro. ¿Podrías verificar y proporcionar el número correcto para ayudarte con el estado de tu pedid

In [63]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

ms = 1759104000000  # tomado de la cabecera
reset_utc = datetime.fromtimestamp(ms/1000, tz=timezone.utc)
reset_local = reset_utc.astimezone(ZoneInfo("America/Bogota"))
print("Reset (UTC):  ", reset_utc)
print("Reset (Bogotá):", reset_local)

ZoneInfoNotFoundError: 'No time zone found with key America/Bogota'